In [ ]:
"""

- Imoveis na periferia de São Paulo são subvalorizados?

- A ideia que norteia o projeto é entender se os imoveis que em raio de 30km do marco 0 da cidade de São Paulo são subvalorizados em comparação ao centro ou bairros nobres
- Para possibilitar uma analise de recomendação de investimento nesses imoveis com base em expectativa de lucro, tempo de retorno e analise de risco do mercado.
- Possiveis interessados na analise exploratoria: Startups do mercado imobiliario, Fundos Imobiliarios, Construtoras, Instituições financeiras que fornecem credito imobiliario

- analise baseada nos imoveis disponiveis nos sites Loft, Quinto Andar e Zap Imoveis

- a primeira missão seria conseguir as informações dos imoveis listados com [id_anuncio, endereco, preco, m2, quartos, banheiro, vaga de garagem, tipo imovel, novo/usado, rating atribuido por mim]
- se possivel obter alguma informação sobre a venda desses imoveis (talvez diario oficial para buscar tranferencias de imoveis)
- com base no endereço começamos a classificar se o imovel está em area periferica, neutra ou nobre
- pós classificação começamos a fazer as comparações de rating e preco de imoveis similares em locais diferentes
- tirar mais ideias sobre a situação imobiliaria de sao paulo
- tentar criar algoritmo que busca oportunidades de lucro 

-- linkar com problemas habitacionais da cidade
-- se possivel realizar uma regressao para comparar o quanto cada variavel impacta no preço do m2/total do imovel e nao se esquecer de adicionar uma variavel para segurança do bairro e outros fatores externos


WebScreaping 
Beautiful soup
Seleniun
Paralelização 

--quando fizer o webscraping procurar as <tags> que contenha apenas a info que eu quero
--para buscar diversas tags diferentes é interessante criar uma função com condicionais para cada type de informação dentro da tag

EXEMPLO
tags = ['p','h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7']
text_list = []
for element in soup.find_all(tags):
    text_list.append(element.text)
print('\n'.join(text_list))


"""

In [ ]:
"""TASK LIST 
- fazer o scraping de uma das variaveis do imovel em uma pagina simples [feito]
- criar uma lista dos links dos imoveis da tela principal



"""

# Imports e acesso ao banco de dados

In [44]:
# Imports que vou utilizar no codigo

from tqdm import tqdm
import pandas as pd
import numpy as np

import requests
from datetime import datetime

from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys



In [ ]:
# Conexão com SQL para no futuro exportar meu Dataframe para SQL

import sqlalchemy
from sqlalchemy import create_engine 

db_server='pymysql'
user='root'
db_port = '3306'
password = 'password'
ip = 'localhost'
db_name = 'conexaopython'
engine = db.create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')
conn = engine.connect()

In [2]:
#localização do chromewebdriver
chrome_path = r'C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\chromedriver'
ser = Service(chrome_path)

In [3]:
#chromewebdriver deve estar aberto para o codigo funcionar
driver = webdriver.Chrome(service=ser)

# Base coleta de dados primeira pagina

In [ ]:
url = 'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'

In [ ]:
driver.get(url)

In [ ]:
driver.current_url

In [ ]:
#metodo do webdriver 'find_element para procurar um elemento dentro da minha URL pelo xpath' 
#peguei o caminho full_xpath do valor no grande box a direita
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/li[2]/p').text

In [ ]:
#peguei o caminho full_xpath do ID do Imóvel nas letras cinzas embaixo das fotos
#será necessário limpar o DataFrame para criar uma chave unica deste ID
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text

In [ ]:
#peguei o caminho full_xpath da rua nas letras cinzas embaixo das fotos 
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text

In [ ]:
#peguei o caminho full_xpath do tipo do imovel nas descrições em preto embaixo das fotos 
#será necessário limpar o tipo com regex, seria uma boa ideia entender os tipos de imovel dentro do site e depois filtrar eles
driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[1]/p').text

In [ ]:
#peguei o caminho full_xpath do titulo nas descrições em preto embaixo das foto para extrair a quantidade de quartos, suites e vagas de gaaragem  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text

In [ ]:
#peguei o caminho full_xpath das descrições para extrair a quantidade de banheiros.  
#será necessário limpar o tipo com regex
driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[5]/p').text

# Func de Coleta uma pagina unica

In [ ]:
url = 'https://loft.com.br/imovel/apartamento-rua-francisco-rossano-vila-prudente-sao-paulo-1-quarto-40m2/ycf79w'
driver.get(url)

In [46]:

df_list = []

def coleta_info(x):
    
    """Essa funcao coleta os dados de um imovel na pagina de anuncio do site Loft por meio do Xpath e salva estes dados dentro de um DataFrame
    Atualmente para a funcao ser executada corretamente é necessario carregar a pagina previamente no navegador de maneira manual
    Eventualmente o site atualiza de um dia para o outro e os caminhos de Xpath devem ser atualizados"""
    
    url = x
    driver.get(url)
    dict_teste = {}
    
    
    try:
        dict_teste['preco_imov'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/li[2]/p').text
    except:
        dict_teste['preco_imov'] = 'Indisponivel'     
    try:    
        dict_teste['id_imov'] = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[9]/span').text
    except:
        dict_teste['id_imov'] = 'Indisponivel'       
    try:
        dict_teste['cidade'] = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[3]/a').text
    except: 
        dict_teste['cidade'] = 'Indisponivel'      
    try:    
        dict_teste['endereco'] = driver.find_element('xpath', '/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[1]/nav/ol/li[7]/a').text
    except: 
        dict_teste['endereco'] = 'Indisponivel'  
    try:    
        dict_teste['titulo_imov'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[1]/div/div/div[2]/h1').text
    except: 
        dict_teste['titulo_imov'] = 'Indisponivel'
    try:    
        dict_teste['vaga'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[4]/p').text
    except: 
        dict_teste['vaga'] = 'Indisponivel'
    try:    
        dict_teste['banheiro'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[2]/div[5]/p').text
    except: 
        dict_teste['banheiro'] = 'Indisponivel'
    try:    
        dict_teste['etc_apart'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[1]/div[3]/div[4]/div/div/p').text 
    except: 
        dict_teste['etc_apart'] = 'Indisponivel'
    try:    
        dict_teste['condominio'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/div[2]/li[2]/div[2]/p').text
    except:
        dict_teste['condominio'] = 'Indisponivel'
    try:
        dict_teste['iptu'] = driver.find_element('xpath','/html/body/div[1]/section/div[2]/div[2]/div/div/div/div/div/div/ul/div[2]/li[3]/div[2]/p').text
    except: 
        dict_teste['iptu'] = 'Indisponivel'
        
    return df_list.append(pd.DataFrame(dict_teste, index=[0]))
    
    #global df_teste
    #df_teste = pd.DataFrame(dict_teste, index=[0]) 
    
    #Quando eu tiro o Global e deixo ele apenas retornando o DataFrame o valor nao sai da função pra nenhum lugar #
    

In [33]:
coleta_info('https://loft.com.br/imovel/apartamento-camburiu-alto-da-lapa-sao-paulo-2-quartos-115m2/1nduya8')

In [34]:
df_list

[     preco_imov         id_imov                    cidade  \
 0  Indisponivel  Imóvel 156UJQ0  Apartamento em São Paulo   
 
                       endereco  \
 0  Rua Bispo Eugênio Demazemod   
 
                                          titulo_imov          vaga  \
 0  Apartamento à venda em Vila Prudente com 52 m²...  Indisponivel   
 
        banheiro     etc_apart    condominio          iptu  
 0  Indisponivel  Indisponivel  Indisponivel  Indisponivel  ,
      preco_imov         id_imov                    cidade      endereco  \
 0  Indisponivel  Imóvel 1NDUYA8  Apartamento em São Paulo  Rua Camburiú   
 
                                          titulo_imov          vaga  \
 0  Apartamento à venda em Alto da Lapa com 115 m²...  Indisponivel   
 
        banheiro     etc_apart    condominio          iptu  
 0  Indisponivel  Indisponivel  Indisponivel  Indisponivel  ]

# Automatização do codigo para rodar em varias paginas.

In [5]:
# coleta de lista de links da home do site

# home_url_loop = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'  #inicia na 2 e vai até a 132

home_url = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'
home_response = requests.get(home_url)
html = home_response.content
soup = BeautifulSoup(html)

# List comprehantion que junta o url da home filtrando dentro da tag <a> todas as tags <href> que iniciam em '/imovel'

lista_links = ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]


NameError: name 'i' is not defined

In [6]:
# For que cria uma lista de todos os links de imoveis listados para sao paulo
# Lista extraida no dia 08/09/2022

lista_links = []

for i in range(1,132):
    home_url = f'https://loft.com.br/venda/imoveis/sp/sao-paulo?pagina={i}'
    home_response = requests.get(home_url)
    html = home_response.content
    soup = BeautifulSoup(html)
    lista_links += ['https://loft.com.br' + tag ['href'] for tag in soup.find_all('a') if tag['href'].startswith('/imovel')]



In [7]:
len(lista_links)

4978

In [96]:
lista_teste = lista_links[3001:]

In [101]:
# Funcao para coletar informacoes de cada imovel dentro da lista de links, estou rodando de 100 em 100 imoveis para evitar um block do site por ip ou um timeout
# após rodar o list(map) tenho que rodar o concat para incluir as informações recem coletadas no df_final

list(tqdm (map(coleta_info, lista_teste)))


1977it [1:42:26,  3.11s/it]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [85]:
from multiprocess import Pool, cpu_count

cpu_count()

12

In [102]:
df_final_final = pd.concat(df_list)
df_final_final

,preco_imov,id_imov,cidade,endereco,titulo_imov,vaga,banheiro,etc_apart,condominio,iptu
0,R$ 1.484.000,Imóvel 1RUQ6KB,Apartamento em São Paulo,Rua dos Pinheiros,"Apartamento à venda em Pinheiros com 68 m², 2 ...",1 vaga,2 Banheiros,Lavabo,R$ 530/mês,R$ 1.830/ano
0,R$ 1.200.000,Imóvel 1NDUYA8,Apartamento em São Paulo,Rua Camburiú,Apartamento à venda em Alto da Lapa com 115 m²...,2 vagas,2 Banheiros,Posição da unidade,R$ 1.195/mês,R$ 391/mês
0,R$ 1.160.000,Imóvel 1UT6HT2,Apartamento em São Paulo,Rua Cunha Gago,"Apartamento à venda em Pinheiros com 60 m², 2 ...",1 vaga,2 Banheiros,Indisponivel,R$ 984/mês,R$ 276/mês
0,R$ 580.000,Imóvel NGVFTG,Apartamento em São Paulo,Rua Adelino de Almeida Castilho,"Apartamento à venda em Tatuapé com 68 m², 2 qu...",1 vaga,2 Banheiros,Posição da unidade,R$ 400/mês,Isento
0,R$ 795.000,Imóvel WBLJ5Z,Apartamento em São Paulo,Rua Araçoiaba,"Apartamento à venda em Saúde com 64 m², 2 quar...",1 vaga,2 Banheiros,Indisponivel,R$ 791/mês,R$ 3.200/ano
...,...,...,...,...,...,...,...,...,...,...
0,R$ 550.000,Imóvel 1MU2S8P,Apartamento em São Paulo,Rua Ausonia,"Apartamento à venda em Tucuruvi com 60 m², 2 q...",2 vagas,2 Banheiros,Indisponivel,R$ 450/mês,R$ 100/ano
0,R$ 355.000,Imóvel NAM32A,Apartamento em São Paulo,Rua Campevas,"Apartamento à venda em Perdizes com 32 m², 1 q...",1 vaga,1 Banheiro,Chuveiro,R$ 630/mês,R$ 160/mês
0,R$ 850.000,Imóvel 5L45RB,Apartamento em São Paulo,Avenida Portugal,"Apartamento à venda em Brooklin com 58 m², 1 q...",1 vaga,1 Banheiro,Lavabo,R$ 1.000/mês,R$ 230/mês
0,R$ 529.000,Imóvel 1UJRSYM,Apartamento em São Paulo,Rua Dona Ana Neri,"Apartamento à venda em Cambuci com 62 m², 3 qu...",1 vaga,2 Banheiros,Vista emparedada,R$ 550/mês,R$ 64/mês


In [92]:
df_final_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1992 entries, 0 to 0
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   preco_imov   1992 non-null   object
 1   id_imov      1992 non-null   object
 2   cidade       1992 non-null   object
 3   endereco     1992 non-null   object
 4   titulo_imov  1992 non-null   object
 5   vaga         1992 non-null   object
 6   banheiro     1992 non-null   object
 7   etc_apart    1992 non-null   object
 8   condominio   1992 non-null   object
 9   iptu         1992 non-null   object
dtypes: object(10)
memory usage: 171.2+ KB


In [117]:
df_final_final.to_csv(r"C:\Users\T-GAMER\Desktop\Ironhack DA\Projeto Final\raw_dataset.csv", encoding = 'latin1')